In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Data Loading and Preprocessing

In [2]:
import pandas as pd

# Path to the dataset
data_path = '/content/drive/MyDrive/heart.csv'

# Load the dataset
data = pd.read_csv(data_path)
data.head()


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
# 1: have heart disease
# 0: does not have a heart disease

Model Training

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features and target
X = data.drop(columns=['output'])
y = data['output']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [6]:
#  Feed-Forward Neural Network
import torch
import torch.nn as nn
import torch.optim as optim

class HeartDiseaseModel(nn.Module):
    def __init__(self, input_size):
        super(HeartDiseaseModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 8)
        self.output = nn.Linear(8, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.output(x))
        return x

def train_model(model, X_train, y_train, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        predictions = model(X_train)
        loss = criterion(predictions, y_train)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch}/{epochs}, Loss: {loss.item()}")

def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        predictions = (predictions > 0.5).float()  # Threshold for binary classification
        accuracy = (predictions == y_test).sum().item() / len(y_test)
        return accuracy


Model Evaluation

In [7]:
# Import necessary libraries
import torch
from sklearn.metrics import accuracy_score

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Initialize model, loss function, and optimizer
model = HeartDiseaseModel(input_size=X_train.shape[1])
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, X_train_tensor, y_train_tensor, criterion, optimizer, epochs=100)

# Evaluate the model
accuracy = evaluate_model(model, X_test_tensor, y_test_tensor)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 0/100, Loss: 0.6806501150131226
Epoch 10/100, Loss: 0.665713906288147
Epoch 20/100, Loss: 0.6493033170700073
Epoch 30/100, Loss: 0.6302018761634827
Epoch 40/100, Loss: 0.6073941588401794
Epoch 50/100, Loss: 0.5803521275520325
Epoch 60/100, Loss: 0.5496413707733154
Epoch 70/100, Loss: 0.5158255696296692
Epoch 80/100, Loss: 0.4809453785419464
Epoch 90/100, Loss: 0.4478929936885834
Test Accuracy: 85.25%
